**Kelvin Fung**<br>
**YSC4216 Machine Learning**<br>
**Assignment 2: Regression Using MAP and Basis Functions**

Link to assignment:
    
https://tanrobby.github.io/teaching/ync_machine_learning/assignment3.html

# Part 1: Full Bayesian Inference 

1. Generate 50 2D-data points using the following function: $y = sin(x^2 + 1)$
2. Add Gaussian random noise to the data
3. Show original curve line and the noisy data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

%matplotlib inline

In [ ]:
# Functions to generate data
def actual_distribution(num_points=50):
    x_act = np.linspace(0, 1, num=num_points)
    y_act = np.sin(np.power(x_act, 2) + 1)
    
    return (x_act, y_act)

def add_noise(y_act, sig=.05):
    mu = 0
    noise = np.random.normal(mu, sig, len(y_act))
    
    return y_act + noise

In [ ]:
x_act, y_act = actual_distribution(50)
y_noise = add_noise(y_act, .05)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(x_act, y_act, '-ro', markersize=3, label='Original curve')
ax.plot(x_act, y_noise, '-gx', linestyle='none', label='Noisy data')

4. Compute w based on full Bayesian Inference (by using basis functions like discussed in class).
5. Display the estimated values of w

In [ ]:
def gaussian_basis(x, miu, sigma):
        return np.exp(-np.power(x-miu, 2) / (2*np.power(sigma, 2)))

In [ ]:
def full_bayesian(x_act, y_noise, M, basis_function, alpha=0, beta=1):
    
    s = 1 / M
    mu_values = [s*i for i in range(1, M)]
    
    phi = np.matrix([[1] + [basis_function(x, mu, s) for mu in mu_values]
                     for x in x_act])
    
    s_n = linalg.inv(np.diag([alpha for _ in range(M)]) + beta*phi.T*phi)
    
    m_n = beta * s_n * phi.T * np.matrix(y_noise).T
    
    y_est = phi*m_n
    
    return m_n, y_est

In [ ]:
w, y_est = full_bayesian(x_act, y_noise, 5, gaussian_basis, .5, 5)

In [ ]:
w

6. Experiment with your code by changing alpha and beta. Discuss the meaning of them with respect to the curve fitting results.

In [ ]:
alpha_values = [0, 0.2, 0.5, 1]
beta_values = [0.5, 1, 5, 20]
plot_num = 1

fig = plt.figure(figsize=[11, 14])
for alpha in alpha_values:
    for beta in beta_values:
        _, y_est = full_bayesian(x_act, y_noise, 5, gaussian_basis, alpha, beta)
        
        ax = fig.add_subplot(4, 4, plot_num)
        plot_num += 1
        
        ax.plot(x_act, y_act, 'ro', markersize=2)   # Original curve
        ax.plot(x_act, y_noise, 'x', markersize=2)  # Noisy data
        ax.plot(x_act, y_est, 'b')                  # Fitted curve
        
        ax.set(title="alpha={}, beta={}".format(alpha, beta))


8. Show your best fitting:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(x_act, y_act, '-ro', markersize=3, label='Original curve')
ax.plot(x_act, y_noise, '-gx', linestyle='none', label='Noisy data')
ax.plot(x_act, y_est, 'b', label='Predicted curve')

9. Increase the value of M, and discuss how it affects the prediction (curve fitting) and the uncertainty of the prediction.

In [ ]:
M_values = [2, 6, 10, 20]
plot_num = 1

fig = plt.figure(figsize=[8, 6])
for M in M_values:
    
    _, y_est = full_bayesian(x_act, y_noise, M, gaussian_basis, .5, 10)
    
    ax = fig.add_subplot(2,2, plot_num)
    plot_num += 1
        
    ax.plot(x_act, y_act, 'ro', markersize=2)   # Original curve
    ax.plot(x_act, y_noise, 'x', markersize=2)  # Noisy data
    ax.plot(x_act, y_est, 'b')                  # Fitted curve

    ax.set(title="M={}".format(M))

At M=2, the predicted curve does not fit well into the underlying curve. At M=10 and M=20, we observe overfitting, where the predicted curve tries to fit the noisy data too closely. At M=6, however, we obtain a predicted curve that fits the underlying curve decently well.

10. Explain how useful p(w|t) is for the training and testing stages.

$p(w|t)$ is useful in the training stage as it tells us how certain or uncertain we are about *w*, which are parameters of the model

# Part 2 - Sequential Bayesian Learning

1. Generate 50 2D-data points using the following function: $y = sin(x^2 + 1)$
2. Add Gaussian random noise to the data

In [ ]:
x_act, y_act = actual_distribution(50)
y_noise = add_noise(y_act, .05)

In [ ]:
def sequential_bayesian(x_act, y_noise, M, basis_function, alpha=0, beta=1):
    
    # Create array to store order in which data is sampled.
    sampling_order = [x for x in range(50)]
    np.random.shuffle(sampling_order)
    
    w_lst = []
    y_est_lst = []
    
    # First set (regular Full Bayesian)
    x_subset = [x_act[i] for i in sampling_order[:5]]
    y_subset = [y_act[i] for i in sampling_order[:5]]
    w, y_est = full_bayesian(x_subset, y_subset, 5, gaussian_basis, .5, 5)
    
    w_lst.append(w)
    y_est_lst.append(y_est)
    
    # Subsequent sets
    
        
    return w_lst, y_est_lst